In [1]:
import random

import numpy as np
import scipy as sp

import scipy.linalg
import scipy.optimize

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_style("white")
sns.set_context("talk")

In [2]:
from markovc import *

In [3]:
from data import *
from inference import *

In [4]:
def dnll(x, data):
    """
    Negative log-likelihood of x given data
    """
    Q = create_rate_matrix_from_vector(x)
    n, _ = Q.shape
    dqiS = np.zeros((n, n))

    
    for (winner, choices), count in data.items():
        winner_idx = [idx for idx, val in enumerate(choices) if winner==val][0]

        Qs = subchain(Q, choices)
        Ps = embedded_jump_chain(Qs)
        piS = equi_dtmc(Ps)
        qiS = equi_ctmc(Qs)[winner_idx]
        
        const = -count/qiS

        choicesIndices = [(c, i) for i, c in enumerate(choices)]
        
        for k in range(len(choicesIndices)):
            for l in range(k+1, len(choicesIndices)):
                i, isub = choicesIndices[k]
                j, jsub = choicesIndices[l]
                dqiS[i, j] += const * equi_deriv_ctmc(Qs, Ps, piS, (isub, jsub))[winner_idx]
    
    # flatten dqiS
    dx = dqiS[np.triu_indices(n, 1)]
    print(x, dx)
    return dx



In [5]:
def fdnll(x, xd, data):
    return nll(xd, data) - nll(x, data)

def fd_equideriv(x, xd):
    Qd = create_rate_matrix_from_vector(xd)
    Q = create_rate_matrix_from_vector(x)
    
    return equi_ctmc(Qd) - equi_ctmc(Q)

In [28]:
alpha = 0.7
x = [alpha, 1-alpha, alpha]
Q = create_rate_matrix_from_vector(x)

P = embedded_jump_chain(Q)
pi = equi_dtmc(P)

Q

array([[-1. ,  0.7,  0.3],
       [ 0.3, -1. ,  0.7],
       [ 0.7,  0.3, -1. ]])

In [29]:
equi_deriv_ctmc(Q, P, pi, (0,1))

array([-0.47819972,  0.36568214,  0.11251758])

In [30]:
data = gen_data(Q, 1000)

In [31]:
x0 = [random.random() for _ in range(len(x))]

res = sp.optimize.minimize(lambda x: nll(x, data), x0, 
                           jac=lambda x: dnll(x, data), 
                           method="L-BFGS-B", bounds=[(0.0001, 0.9999) for _ in range(len(x0))])
# maxits = 100
# x = x0
# alpha = 1e-4
# for k in range(maxits):
#     x = x + alpha * dnll(x, data)
#     x = np.clip(x, 0.1, 0.9)

print(res)

[ 0.91724519  0.67940322  0.1389017 ] [ 1303.30023956   496.30962621 -1649.05652573]
[  1.00000000e-04   1.00000000e-04   9.99900000e-01] [-2729699.01744596 -1349744.97799797   779906.96349916]
[ 0.50421367  0.37348258  0.52664776] [-229.7351537    29.24514682 -192.99096028]
[ 0.56805103  0.36532991  0.58361458] [-157.9366031    33.3971632  -131.67373358]
[ 0.71386275  0.30402066  0.7082297 ] [  8.47138501 -25.05709809   8.33470464]
[ 0.70459458  0.32808999  0.70208123] [ -1.83733123  10.23492447   3.06584169]
[ 0.70525257  0.32119951  0.70104954] [-1.48424465 -0.15027765  0.63122933]
[ 0.70596247  0.32114971  0.7011216 ] [-0.46862501 -0.14192465  0.64555981]
[ 0.70624077  0.32121415  0.70083588] [-0.02256603 -0.04872508  0.19567295]
[ 0.70625117  0.32125349  0.70070913] [ 0.01220709 -0.00307883  0.01129528]
[ 0.70624262  0.32125675  0.70070059] [  1.16487855e-03  -6.49371935e-05   7.82457973e-05]
        x: array([ 0.70624262,  0.32125675,  0.70070059])
  message: b'CONVERGENCE: REL_R

In [32]:
xopt = res['x']
Qhat = create_rate_matrix_from_vector(xopt)

In [33]:
Qhat

array([[-1.02749936,  0.70624262,  0.32125675],
       [ 0.29375738, -0.99445798,  0.70070059],
       [ 0.67874325,  0.29929941, -0.97804266]])

In [34]:
nll(xopt, data)

713.53036832096109

In [35]:
nll(x, data)

713.9235423376158

In [ ]:
Phat = embedded_jump_chain(Qhat)
pihat = equi_ctmc(Qhat)

In [ ]:
equi_deriv_ctmc(Qhat, Phat, pihat, (2, 1))

In [ ]:
nll(x0, data), dnll(x0, data)

In [ ]:
nll(xopt, data), dnll(xopt, data)

In [ ]:
for (winner, choices), count in gen_data(Q, 10).items():
    print(winner, choices, count)

In [ ]:
x = list(range(5))

In [ ]:
choices = (2 ,5)

In [ ]:
[(i, x) for i, x in enumerate(choices)]

In [ ]:
np.triu_indices(3, 1)

In [ ]:
Q[np.triu_indices(3, 1)]

In [ ]:
Q

In [ ]:
dnll(x0, data)

In [ ]:
dnll(xopt, data)

In [ ]:
sp.optimize.minimize?

In [ ]:
nll(x0, data)

In [ ]:
nll(xopt, data)

In [ ]:
dnll(xopt, data)

In [ ]:
delta = 1e-8

ind = np.array([1, 0, 0])

x = x0
xp = x + delta * ind

(dnll(x, data) @ ind - fdnll(x, xp, data) / delta) / (dnll(x, data) @ ind)

In [ ]:
xopt

In [ ]:
create_rate_matrix_from_vector(x0)

In [ ]:
create_rate_matrix_from_vector(x1)

In [ ]:
nll(x0, data)

In [ ]:
nll(x1, data)

In [ ]:
fd_equideriv(x, xp)/delta

In [ ]:
Q = create_rate_matrix_from_vector(x)
P = embedded_jump_chain(Q)
pi = equi_dtmc(P)

equi_deriv_ctmc(Q, P, pi, (0, 1))

In [ ]:
x

In [ ]:
xp